# Example 02: SoQL Queries

Constructing custom queries to conserve bandwith and computational resources


## Setup


In [1]:
import os
# Note: we don't need Pandas
# Filters allow you to accomplish many basic operations automatically

from sodapy import Sodapy

## Find Some Data

As in the first example, I'm using the [NYC campaign contribution dataset](https://data.cityofnewyork.us/City-Government/Campaign-Contributions/rjkp-yttg/about_data).


In [2]:
socrata_domain = "data.cityofnewyork.us"
socrata_dataset_identifier = "rjkp-yttg"

# If you choose to use a token, run the following command on the terminal (or add it to your .bashrc)
# $ export SODAPY_APPTOKEN=<token>
socrata_token = os.environ.get("SODAPY_APPTOKEN")

In [3]:
client = Sodapy(socrata_domain, socrata_token)

## Use Metadata to Plan Your Query

You've probably looked through the column names and descriptions in the web UI,
but it can be nice to have them right in your workspace as well.


In [4]:
metadata = client.get_metadata(socrata_dataset_identifier)
metadata

{'resource': {'name': 'Campaign Contributions',
  'id': 'rjkp-yttg',
  'resource_name': None,
  'parent_fxf': [],
  'description': 'Contributions to election campaigns.',
  'attribution': 'Campaign Finance Board (CFB)',
  'attribution_link': None,
  'contact_email': None,
  'type': 'dataset',
  'updatedAt': '2025-02-04T22:38:26.000Z',
  'createdAt': '2021-02-16T00:52:16.000Z',
  'metadata_updated_at': '2025-02-04T22:38:26.000Z',
  'data_updated_at': '2025-02-04T22:06:26.000Z',
  'page_views': {'page_views_last_week': 12,
   'page_views_last_month': 57,
   'page_views_total': 973,
   'page_views_last_week_log': 3.700439718141092,
   'page_views_last_month_log': 5.857980995127572,
   'page_views_total_log': 9.927777962082342},
  'columns_name': ['CITY',
   'PURPOSECD',
   'INTEMPST',
   'INT_C_CODE',
   'INTEMPCITY',
   'INTEMPSTNM',
   'INTEMPNAME',
   'RECIPID',
   'CANCLASS',
   'CANDFIRST',
   'COMMITTEE',
   'FILING',
   'INTCITY',
   'REFNO',
   'INTSTRNM',
   'DATE',
   'REFUNDDAT

In [5]:
metadata["resource"]["columns_name"]

['CITY',
 'PURPOSECD',
 'INTEMPST',
 'INT_C_CODE',
 'INTEMPCITY',
 'INTEMPSTNM',
 'INTEMPNAME',
 'RECIPID',
 'CANCLASS',
 'CANDFIRST',
 'COMMITTEE',
 'FILING',
 'INTCITY',
 'REFNO',
 'INTSTRNM',
 'DATE',
 'REFUNDDATE',
 'INTSTRNO',
 'INTERMNAME',
 'INTERMNO',
 'PAY_METHOD',
 'MATCHAMNT',
 'NAME',
 'C_CODE',
 'STRNO',
 'STRNAME',
 'APARTMENT',
 'AMNT',
 'BOROUGHCD',
 'EMPSTATE',
 'EMPCITY',
 'STATE',
 'INTOCCUPA',
 'EMPNAME',
 'EMPSTRNO',
 'EMPSTRNAME',
 'INTZIP',
 'INTST',
 'SEQUENCENO',
 'OCCUPATION',
 'PREVAMNT',
 'ADJTYPECD',
 'ELECTION',
 'EXEMPTCD',
 'INTAPTNO',
 'INTEMPSTNO',
 'PAGENO',
 'RECIPNAME',
 'SCHEDULE',
 'ZIP',
 'SEG_IND',
 'RR_IND',
 'CANDMI',
 'OFFICECD']

## Efficiently Query for Data


### Restrict rows to above-average donations

Using [SoQL's `avg()` function](https://dev.socrata.com/docs/functions/avg).


In [6]:
avg_result = client.get(socrata_dataset_identifier, select="avg(amnt)")
avg_result

[{'avg_amnt': '454.8782932298324902'}]

Use the [`where`](https://dev.socrata.com/docs/queries/where) argument to filter the data before downloading it. How many campaign contributions have there been in the past year that are more than double the average?


In [7]:
from datetime import datetime

avg = float(avg_result[0]["avg_amnt"])
current_year = datetime.now().year

results = client.get(
    socrata_dataset_identifier,
    where=f"amnt >= ({avg} * 2) and date_extract_y(date) == {current_year}",
)
len(results)

611

### Perform basic operations

You can further simplify the download by specifying just the columns you want. It can also be valuable to have results in order, so that you can quickly grab the largest or smallest. You can even accomplish some basic analytics operations like finding [sums](https://dev.socrata.com/docs/functions/sum.html).

If you're planning on doing further processing, note that the numeric outputs are strings by default.


In [8]:
results = client.get(
    socrata_dataset_identifier,
    select="sum(amnt), recipname",
    group="recipname",
    order="sum(amnt) DESC",
)
results

[{'sum_amnt': '108375000', 'recipname': 'Bloomberg, Michael R'},
 {'sum_amnt': '84155868', 'recipname': 'Bloomberg'},
 {'sum_amnt': '19082335.99', 'recipname': 'de Blasio, Bill'},
 {'sum_amnt': '14288722.48', 'recipname': 'Adams, Eric L'},
 {'sum_amnt': '11232542', 'recipname': 'Green'},
 {'sum_amnt': '11083662.08', 'recipname': 'Catsimatidis, John A'},
 {'sum_amnt': '10715128.70', 'recipname': 'Stringer, Scott M'},
 {'sum_amnt': '10709864', 'recipname': 'Ferrer'},
 {'sum_amnt': '10694368.19', 'recipname': 'Spitzer, Eliot L'},
 {'sum_amnt': '10041871.86', 'recipname': 'McGuire, Raymond J'},
 {'sum_amnt': '9356429.63', 'recipname': 'Thompson, Jr., William C'},
 {'sum_amnt': '8119591.55', 'recipname': 'Quinn, Christine C'},
 {'sum_amnt': '7181923', 'recipname': 'Hevesi'},
 {'sum_amnt': '7111741.72', 'recipname': 'Liu, John C'},
 {'sum_amnt': '6977004.20', 'recipname': 'New Start NYC'},
 {'sum_amnt': '6960000.00', 'recipname': 'Jobs for New York, Inc.'},
 {'sum_amnt': '6920000', 'recipnam

### Break download into managable chunks

Sometimes you do want all the data, but it would be too big for one download.

By default, all queries have a limit of 1000 rows, but you can manually set it
higher or lower. If you want to loop through results, just use `offset`


In [9]:
results = client.get(
    socrata_dataset_identifier,
    limit=6,
    select="name, amnt",
    where="name is not null",
)
results

[{'name': "THE SPACE AN ARTISTS' COLLEC", 'amnt': '50'},
 {'name': '. HASSAD, ABDOOL K', 'amnt': '100'},
 {'name': '?, SAUL', 'amnt': '25'},
 {'name': '1-10 INDUSTRY ASSOCIATES LLC', 'amnt': '4500'},
 {'name': '100  CHURCH LLC', 'amnt': '2000'},
 {'name': '100 MOSHOLU PARKWAY ASSC. LLC', 'amnt': '500'}]

To retrieve _all_ records, you'd use [`get_all()`](../README.md#get_alldataset_identifier-content_typejson-kwargs).


### Query strings

All of the queries above were made with method parameters,
but you could also pass all the parameters at once in a
SQL-like format


In [10]:
query = """
select
    name,
    amnt
where
    amnt > 1000
    and amnt < 2000
limit
    5
"""

results = client.get(socrata_dataset_identifier, query=query)
results

[{'name': 'Freudenheim, Ellen', 'amnt': '1000.18'},
 {'name': 'Kurland, Yetta G', 'amnt': '1000.31'},
 {'name': 'Chain, Heidi H', 'amnt': '1000.94'},
 {'name': 'SONI, KRISHNAKUMAR L', 'amnt': '1001'},
 {'name': 'SUGIMOTO, YUJI', 'amnt': '1001'}]

### Free text search

My brother just got a dog named Slider, so we were curious about how many other New York City dogs had that name.

Searches with `q` match anywhere in the row, which allows you to quickly search through data with several free text columns of interest.


In [11]:
nyc_dogs_domain = "data.cityofnewyork.us"
nyc_dogs_dataset_identifier = "nu7n-tubp"

nyc_dogs_client = Sodapy(nyc_dogs_domain, socrata_token)
results = nyc_dogs_client.get(
    nyc_dogs_dataset_identifier, q="Slider", select="animalname, breedname"
)
results

[{'animalname': 'SLIDER', 'breedname': 'American Pit Bull Terrier/Pit Bull'},
 {'animalname': 'SLIDER', 'breedname': 'Cavalier King Charles Spaniel'},
 {'animalname': 'SLIDER', 'breedname': 'Shih Tzu'},
 {'animalname': 'SLIDER', 'breedname': 'Wheaton Terrier'},
 {'animalname': 'SLIDER', 'breedname': 'Cavalier King Charles Spaniel'},
 {'animalname': 'SLIDER', 'breedname': 'Shih Tzu'},
 {'animalname': 'SLIDER', 'breedname': 'Golden Retriever'},
 {'animalname': 'SLIDER', 'breedname': 'Shih Tzu'},
 {'animalname': 'SLIDER', 'breedname': 'Unknown'},
 {'animalname': 'SLIDER', 'breedname': 'Shih Tzu'},
 {'animalname': 'SLIDER', 'breedname': 'Shih Tzu'},
 {'animalname': 'SLIDER', 'breedname': 'Dachshund, Long Haired Miniature'},
 {'animalname': 'SLIDER', 'breedname': 'Dachshund, Long Haired Miniature'},
 {'animalname': 'SLIDER', 'breedname': 'Shih Tzu'}]

# Going Further

There's plenty more to do! Check out [Queries using SODA](https://dev.socrata.com/docs/queries/) for additional functionality
